# ACCESS-MOPPeR Getting Started

## Set up configuration

When you first import `access_mopper` in a Python environment, the package will automatically create a `user.yml` file in your home directory (`~/.mopper/user.yml`).  

During this initial setup, you will be prompted to provide some basic information, including:  
- Your name  
- Your email address  
- Your work organization
- Your ORCID

This information is stored in `user.yml` and will be used as global attributes in the files generated during the CMORisation process. This ensures that each CMORised file includes metadata identifying who performed the CMORisation, allowing us to track data provenance and follow up with the responsible person if needed.

In [3]:
import access_mopper as mop
import dask.distributed as dask


Loaded Configuration:
Creator Name: Romain Beucher
Organisation: ACCESS-NRI
Creator Email: romain.beucher@anu.edu.au
Creator URL: 0000-0003-3891-5444


In [4]:
client = dask.Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 15.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33245,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.06 GiB
Comm: tcp://127.0.0.1:33595,Total threads: 1
Dashboard: http://127.0.0.1:37043/status,Memory: 1.25 GiB
Nanny: tcp://127.0.0.1:46317,


In [5]:
from access_mopper import ACCESS_ESM_CMORiser

In [6]:
parent_experiment_config = {
        "parent_experiment_id": "piControl",
        "parent_activity_id": "CMIP",
        "parent_source_id": "ACCESS-ESM1-5",
        "parent_variant_label": "r1i1p1f1",
        "parent_time_units": "days since 0001-01-01 00:00:00",
        "parent_mip_era": "CMIP6",
        "branch_time_in_child": 0.0,
        "branch_time_in_parent": 54786.0,
        "branch_method": "standard"
}

In [7]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths="../tests/data/esm1-6/atmosphere/aiihca.pa-096110_mon.nc", 
    compound_name="Amon.rsds",
    experiment_id="historical",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config)

In [8]:
cmoriser.run()

CMORised output written to rsds_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_096110-096110.nc


In [9]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths="../tests/data/esm1-6/ocean/ocean-2d-surface_temp-1monthly-mean-ym_0966_01.nc", 
    compound_name="Omon.tos",
    experiment_id="historical",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config)

In [10]:
cmoriser.run()

CMORised output written to tos_Omon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_096601-096612.nc
